In [10]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd

import time
import os
import csv
import json
import re
MOSTPOPULAR = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

In [11]:
def getFilmInfor(url):
    # instantiate a chrome options object so you can set the size and headless preference 
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument('blink-settings=imagesEnabled=false')  
    
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads 
    # and put it in the current directory 
    chrome_driver = os.getcwd() + "\\chromedriver2_win32" + "\\chromedriver.exe" 

    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
    driver.get(url)
    # get all id and title
    filmInfor = list()
    for film in driver.find_elements_by_xpath("//tbody[@class='lister-list']/tr"):
        title = film.find_element_by_xpath("td[2]/a").text
        filmId = film.find_element_by_xpath("td[4]/div").get_attribute("data-titleid")
        try:
            avgStars = film.find_element_by_xpath("td[3]").text
        except:
            avgStars = ''
        try:
           year = film.find_element_by_xpath("td[2]/span").text 
           year = re.findall(r"\d+",year)[0] #get the year
        except:
            year = ''
        filmInfor.append([filmId,title,year,avgStars])
    print(len(filmInfor))
    driver.quit()
    return filmInfor

In [12]:
def getReviews(filmId):
    chrome_options = Options() 
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--window-size=1920x1080") 
    chrome_options.add_argument('blink-settings=imagesEnabled=false') 
    url = "https://www.imdb.com/title/"+filmId+"/reviews"

    chrome_driver = os.getcwd() + "\\chromedriver2_win32" + "\\chromedriver.exe" 
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)
    wait = WebDriverWait(driver,10)

    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    while True:
        try:
            driver.find_element_by_css_selector("button#load-more-trigger").click()
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".ipl-load-more__load-indicator")))
            soup = BeautifulSoup(driver.page_source, 'lxml')
        except Exception:break
 
    reviews = list()#[stars,text]
    for review in soup.find_all(class_='imdb-user-review'):
        if review.find("span",class_="spoiler-warning")==None:
            try:
                text = review.find(class_='show-more__control').get_text(strip=True)
                stars = review.find(class_="rating-other-user-rating").get_text(strip=True).split("/")[0]
                reviews.append([stars,text])
            except:
                pass
    driver.quit()
    return reviews

In [13]:
%%time
csvFile = open("most_popular_films.csv", 'wt', newline='\n', encoding='utf-8')
writer = csv.writer(csvFile)
header = ['id','name','year','avg_stars','stars','review']
writer.writerow(header)
for filmInfor in getFilmInfor(MOSTPOPULAR):
    temp = list()
    for review in getReviews(filmInfor[0]):
        temp.append(filmInfor+review)
    writer.writerows(temp)
    print(filmInfor[1]+"is done")
    print("-------------------------")
csvFile.close()

100
Aquamanis done
-------------------------
Underwateris done
-------------------------
Gisaengchungis done
-------------------------
Unchartedis done
-------------------------
Itis done
-------------------------
Jokeris done
-------------------------
Friday the 13this done
-------------------------
The Suicide Squadis done
-------------------------
Kadaveris done
-------------------------
The Wolf of Wall Streetis done
-------------------------
The Secrets We Keepis done
-------------------------
The Shawshank Redemptionis done
-------------------------
News of the Worldis done
-------------------------
I'm Thinking of Ending Thingsis done
-------------------------
Yes, God, Yesis done
-------------------------
The Mortuary Collectionis done
-------------------------
A Nightmare on Elm Streetis done
-------------------------
Addams Family Valuesis done
-------------------------
The Addams Familyis done
-------------------------
The Silence of the Lambsis done
------------------------

In [16]:
df = pd.read_csv("most_popular_films.csv")

In [18]:
df["stars"].describe()

count     97192
unique       11
top          10
freq      35777
Name: stars, dtype: object